# scratch work

In [1]:
import jax

jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp

from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.jplhorizons import Horizons

from jorbit.utils.horizons import (
    horizons_bulk_vector_query,
    horizons_bulk_astrometry_query,
)

t0 = Time("2024-12-24T00:00:00", scale="utc")